### Can we identify the subject based on their physiological signals and self-reports? ###

In [3]:
# IMPORTING MODULES
import glob
import importlib
import matplotlib.pyplot as plt
import numpy as np
import os
cvx_path = os.path.abspath(os.path.join('..', '..', 'cvxEDA', 'src'))
module_path = os.path.abspath(os.path.join('..', '..', 'src'))
sys.path.append(module_path)
import pandas as pd
import random
import scipy.signal as ss
import sys

import tools.data_reader_apd as dr
import tools.display_tools as dt
import tools.preprocessing as preprocessing
import train

from scipy.fft import fft, fftfreq, fftshift
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split, cross_val_score, RepeatedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import normalize

import cvxopt.solvers
cvxopt.solvers.options['show_progress'] = False

import warnings
warnings.filterwarnings(
    "ignore", 
    category=RuntimeWarning
)

In [30]:
# LOAD TRAIN AND TEST DATA
importlib.reload(train)
importlib.reload(preprocessing)
importlib.reload(dr)
importlib.reload(dt)

metrics = [
    "bpm", 
    "rmssd", 
    "hf_rr", 
    "lf_rr", 
    "ibi", 
    "mean_SCL", 
    "SCR_rate"
]

model_phases = [
    dr.Phases.BASE_REST,
    dr.Phases.BUG_RELAX, dr.Phases.BUG_ANTICIPATE, dr.Phases.BUG_EXPOSURE, dr.Phases.BUG_BREAK,
    dr.Phases.SPEECH_RELAX, dr.Phases.SPEECH_ANTICIPATE, dr.Phases.SPEECH_EXPOSURE, dr.Phases.SPEECH_BREAK
]

if type(model_phases) != list:
    model_phases = [model_phases]

anxiety_label_type = "Gender"
x, y = train.Train_APD.get_apd_data_ranking(metrics, model_phases, verbose=False, anxiety_label_type=anxiety_label_type)
# drop subjects with noisy data
x = x[x['subject'] != 84.0]
y = y[y['subject'] != 84.0]
x = x[x['subject'] != 8.0]
y = y[y['subject'] != 8.0]

# print(x.head())

# 0-1 scaling
for i in range(3, len(x.columns)):
    data_col = x[x.columns[i]]
    data_col = (data_col - data_col.min())/(data_col.max() - data_col.min())
    x[x.columns[i]] = data_col

# print(x.head())
# print(y.head())
# print(y.loc[:, "label"].value_counts())

models = {
    "SVM": SVC(), 
    "KNN": KNeighborsClassifier()
}

results = {"SVM": [], "KNN": []}
num_iters = 10
for _ in range(num_iters):
    out = train.train_predict(models, x, y, show_classification=False)
    for model_name in results:
        results[model_name].append(out[model_name])

for model_name in results.keys():
    print(f"{model_name} accuracy over {num_iters} rounds: {np.mean(results[model_name])}")

SVM accuracy over 10 rounds: 0.6619047619047619
KNN accuracy over 10 rounds: 0.6079365079365079
